#Importo librerie

In [1]:
import pandas as pd
import numpy as np
import concurrent.futures
import time
from requests.exceptions import ReadTimeout

In [2]:
!pip install -U -q PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Autenticazione Spotify API

In [ ]:
!pip uninstall spotipy

In [4]:
!pip install spotipy

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 153kB 11.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
# autenticazione Spotify API con spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
auth_manager = SpotifyClientCredentials(client_id='caf57b996b464996bff50ab59186f265', client_secret='0bfcdbff8015426cae855b56b692f69b')
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10)

#Importo dataset

In [10]:
# autenticazione google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [11]:
# importo quarta parte del dataset --> 7087 datapoints

drive.CreateFile({'id':'1YAAvPUaPeBIddkuVzWgw7fhPPcT2uJTY'}).GetContentFile('billboard_dataset_unique.csv')
df_billboard = pd.read_csv("billboard_dataset_unique.csv").drop('Unnamed: 0',axis=1).iloc[21261:]

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)


# df_billboard.iloc[:7087]

# df_billboard.iloc[7087:14174]

# df_billboard.iloc[14174:21261]

# df_billboard.iloc[21261:]

In [12]:
df_billboard.head()

,title,artist,weeks
21261,Long Lonesome Highway,Michael Parks,12
21262,Ticket To Ride,Carpenters,12
21263,"My Wife, The Dancer",Eddie & Dutch,7
21264,You Make Me Real/Roadhouse Blues,The Doors,6
21265,So Excited,B.B. King,6


In [13]:
df_billboard.shape

(7088, 3)

#Definizione funzioni

In [14]:
def print_exec_time(start):
  print("Esecuzione completata in %.4f secondi" % (time.perf_counter()-start))

In [15]:
# funzione che effettua ricerca con Spotify API considerando i casi in cui nel campo 'artist' siano presenti più artisti (featuring)

def search_fix(artist, title):
  artist_separators = ['%%%', ' Featuring', ' featuring', ' feat.', ' Feat.', ' feat', ' Feat', ' &', ' x', ' X', ' with', ' With', ', ', '/', ' duet', ' Duet', '+', ' and']
  title_separators = ['%%%', ' (']
  title_fix = ["%%%", "'s", "'"]

  id = None

  for x in artist_separators:
    for y in title_separators:
      for z in title_fix:
        try:
          id = sp.search(q='artist:'+artist.split(x)[0]+' track:'+title.split(y)[0].replace(z, ''), type='track', limit=1)['tracks']['items'][0]['id']
        except IndexError:
          pass
        if(id != None):
          break
      if(id != None):
        break
    if(id != None):
      break

  return id

In [16]:
# funzione che prendendo una singola riga del Billboard dataset restituisce una lista con id, artista e titolo
# --> in caso di errore l'id viene impostato a None

def get_id(row):

  artist = row[1]
  title = row[0]

  print("fetching id for %s by %s ..." % (title, artist))

  try:
    try:
      id = sp.search(q='artist:'+artist+' track:'+title, type='track', limit=1)['tracks']['items'][0]['id']
    except IndexError:
      id = search_fix(artist, title)
  except ReadTimeout:
    id = None
      
  if(id == None):
    print('--> [error] %s by %s' % (title, artist))

  return [id, artist, title]

In [17]:
# funzione che, preso un id, restituisce un array con le features (audio e non) della traccia corrispondente

def get_features(id):
  print("fetching features for id: %s" % id)

  # audio features
  danceability = []
  energy = []
  key = []
  loudness =[]
  mode = []
  speechiness = []
  acousticness = []
  instrumentalness = []
  liveness = []
  valence = []
  tempo = []
  duration_ms = []

  audio_features_array = [danceability, energy, key, loudness, mode, speechiness,
      acousticness, instrumentalness, liveness, valence, tempo, duration_ms]

  # altre features
  release_date = []
  explicit = []

  release_date.append(sp.track(id)['album']['release_date'])
  explicit.append(sp.track(id)['explicit'])

  audio_features = sp.audio_features(id)[0]

  try:
    # rimuovo campi non necessari
    to_remove = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature']
    for rmv in to_remove:
        audio_features.pop(rmv)

    for i, feature in enumerate(audio_features.keys()):
        audio_features_array[i].append(audio_features[feature])

  except AttributeError:
    print("--> [error] id = %s" % id)

    for i in range(12):
        audio_features_array[i].append(None)


  audio_features_array.append(release_date)
  audio_features_array.append(explicit) 

  return audio_features_array

#Integrazione dataset

##Recupero gli id del Billboard dataset

In [18]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_id, df_billboard.values.tolist())
  
  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

Output streaming troncato alle ultime 5000 righe.
fetching id for Keep On Running by The Spencer Davis Group ...
fetching id for (I'm Just A) Fool For You by Gene Chandler ...
fetching id for The Boogaloo Party by The Flamingos ...
--> [error] Uptight (Everything's Alright) by The Jazz Crusaders
fetching id for Baby I Need You by The Manhattans ...
fetching id for Sharing You by Mitty Collier ...
fetching id for I Spy (For The FBI) by Jamo Thomas & His Party Brothers Orchestra ...
--> [error] The Boogaloo Party by The Flamingos
fetching id for When Liking Turns To Loving by Ronnie Dove ...
fetching id for Don't Mess With Bill by The Marvelettes ...
fetching id for What Now My Love by Sonny & Cher ...
fetching id for At The Scene by The Dave Clark Five ...
fetching id for Uptight (Everything's Alright) by Stevie Wonder ...
fetching id for My Love by Petula Clark ...
fetching id for Crying Time by Ray Charles ...
--> [error] I Spy (For The FBI) by Jamo Thomas & His Party Brothers Orchest

In [19]:
# creo backup del billboard dataset
df_billboard_bak = df_billboard.copy()

In [20]:
# inserisco gli id ottenuti in una nuova colonna nel df_billboard
ids = np.array(output)[:,0]
df_billboard.insert(0, 'id', ids)

In [21]:
# calcolo percentuale di canzoni trovate
found_id = df_billboard.id.count()
x = (found_id / df_billboard.title.count()) * 100
print("Found ids = %d%%" % x)

Found ids = 82%


In [22]:
# esporto su google drive
from google.colab import drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+ids_3.csv')

Mounted at /content/drive


##Recupero audio features del Billboard dataset

In [23]:
# reimporto dataset billboard (con ids) + dataset principale
"""
drive.CreateFile({'id':'1fZzuYu-HXKP9HUeio-FL9P4eNygOQ0qq'}).GetContentFile('billboard+ids_0.csv') 
df_billboard = pd.read_csv("billboard+ids_0.csv").drop('Unnamed: 0',axis=1)

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)
"""

'\ndrive.CreateFile({\'id\':\'1fZzuYu-HXKP9HUeio-FL9P4eNygOQ0qq\'}).GetContentFile(\'billboard+ids_0.csv\') \ndf_billboard = pd.read_csv("billboard+ids_0.csv").drop(\'Unnamed: 0\',axis=1)\n\ndrive.CreateFile({\'id\':\'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0\'}).GetContentFile(\'dataset2_X_billboard.csv\')\ndf_songs = pd.read_csv("dataset2_X_billboard.csv").drop(\'Unnamed: 0\',axis=1)\n'

In [24]:
# elimino valori nulli (= id non trovati)
df_billboard = df_billboard.dropna()

# creo lista con gli id del dataset billboard
ids = list(df_billboard.id.array)

In [25]:
# creo lista degli id che non sono presenti nel dataset principale
time_0 = time.perf_counter()
ids_new = [id for id in ids if id not in list(df_songs.id.array)]
print_exec_time(time_0)

Esecuzione completata in 1374.0148 secondi


In [26]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_features, ids_new)

  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

fetching features for id: 3PyQV3cDjV5tEJGpYYH2K1
fetching features for id: 7qHGRefOGiaPqrG4IEckcv
fetching features for id: 1PSfY3lSwBD888PZW0s5JH
fetching features for id: 1er2tyXb4iw23SfZxb1FW1
fetching features for id: 6C9MYPkHrESo8n3KnapaR9
fetching features for id: 2kLrFBgVs9BsyNQ6EDc5LH
fetching features for id: 4Ht8wlFBxdNiQQSdWTBOik
fetching features for id: 4Dxm37KSWN6xXTn98ddrGp
fetching features for id: 4DvqAY9mzeTjQUNvsROwji
fetching features for id: 6nd9sBgQj3ABs1ZmNrStoR
fetching features for id: 26R4HdOOnj4zYGl8lbYW9j
fetching features for id: 6IjvCFBNprUHhlamLey2Tb
fetching features for id: 7K40sZ0ZV6vGjzSso2nFyr
fetching features for id: 3NcbI0iX8VQyR9D3qsmNjl
fetching features for id: 3HZeaD9anHDrTM1AW5PY7y
fetching features for id: 4K0PEpyhPdemgNj1CRPGOj
fetching features for id: 5raayWOkxcm1pt5p59xN8k
fetching features for id: 723SZaMK90mxb59BZj3Oig
fetching features for id: 3YBzEkdpzpp0NomRIl56da
fetching features for id: 1sEImexIhca9b7pUcpObsC
fetching features fo

#Inserisco audio features nel dataset Billboard

In [27]:
num_datapoints = np.array(output).shape[0]

output = np.array(output).reshape((num_datapoints,14))

In [28]:
# creo backup df_billboard
df_billboard_bak = df_billboard.copy()

# filtro dataset billboard tenendo solo id nell'array 'ids_new', quindi quelli che non sono presenti nel dataset principale
df_billboard = df_billboard[df_billboard.id.isin(ids_new)]

In [29]:
to_insert = ['danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',  
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms',
    'release_date',
    'explicit']

for i, col in enumerate(output.T):
    df_billboard.insert(4, to_insert[i], col)

In [30]:
# converto colonna 'release_date' in tipo datetime
df_billboard.release_date = pd.to_datetime(df_billboard.release_date,format="%Y-%m-%d",exact=False)

In [31]:
# inserisco colonna 'year'
year = df_billboard['release_date'].apply(lambda x: int(x.year))
df_billboard.insert(6, 'year', year)

In [32]:
# inserisco colonna 'popularity' --> nb: inizializzo a 0 perchè verrà rimossa
df_billboard.insert(17, 'popularity', np.zeros(df_billboard.shape[0]))

In [33]:
# inserisco colonna 'hit'
hit = np.ones(df_billboard.shape[0])
df_billboard.insert(3, 'hit', hit)
df_billboard.hit = df_billboard.hit.apply(int)

In [34]:
df_billboard.head()

,id,title,artist,hit,weeks,explicit,release_date,year,duration_ms,tempo,valence,liveness,instrumentalness,acousticness,speechiness,mode,loudness,key,popularity,energy,danceability
21261,7qHGRefOGiaPqrG4IEckcv,Long Lonesome Highway,Michael Parks,1,12,False,2011-10-01,2011,128280,85.112,0.343,0.12,0.0635,0.894,0.0412,1,-13.534,2,0.0,0.263,0.605
21262,3PyQV3cDjV5tEJGpYYH2K1,Ticket To Ride,Carpenters,1,12,False,2018-12-07,2018,250173,179.534,0.251,0.121,0,0.468,0.0292,1,-7.502,10,0.0,0.435,0.248
21266,1PSfY3lSwBD888PZW0s5JH,Deeper (In Love With You),The O'Jays,1,7,False,2008-06-10,2008,159293,169.362,0.902,0.344,0,0.208,0.0305,1,-11.118,3,0.0,0.562,0.575
21267,1er2tyXb4iw23SfZxb1FW1,Grover Henson Feels Forgotten,Bill Cosby,1,3,False,1992-01-01,1992,220800,89.211,0.35,0.137,5.63e-05,0.568,0.0397,1,-13.36,0,0.0,0.417,0.237
21270,6C9MYPkHrESo8n3KnapaR9,Patch Of Blue,Frankie Valli & The 4 Seasons,1,2,False,2007-06-04,2007,185013,115.379,0.733,0.0477,0,0.264,0.0409,1,-7.511,10,0.0,0.727,0.498


#Esporto

In [35]:
# esporto in google drive
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+features_3.csv')

Mounted at /content/drive
